In [1]:
#import libraries and YOLO model using internal python package for YOLO
import cv2
import numpy as np
import random

#edgeCase-1
#occluded condition
#description: Objects partially hidden behind other objects.
#example: A pedestrian partially hidden by a parked car in an image.
#effect: YOLO might miss the object or provide incorrect bounding boxes
def apply_occlusion(image):
    "The function simulates occlusion by covering part of the image with a rectangle"
    #extract the height and width of the image, ignores channel
    h, w = image.shape[:2] 
    #generate the coordinates of a rectangle randomly between (0 to w/2, 0 to h/2), these value can be changed
    top_left = (random.randint(100, w//2), random.randint(100, h//2)) 
    #add the random width and height (i.e., between (50,200) these value can be changed) to the previously generated random 
    #points to get the bottom right of a rectangle
    bottom_right = (top_left[0] + random.randint(500, 800), top_left[1] + random.randint(500, 800)) 
    #black rectangle to occlude
    cv2.rectangle(image, top_left, bottom_right, (0, 0, 0), -1) 
    return image

#edgeCase-2
#extreme lighting conditions i.e., brightening and darkening
#description: Objects in low-light or high-light conditions.
#example: A person standing in the shadow of a building or under direct sunlight.
#effect: The model might fail to detect objects in very bright or very dark areas of the image
def apply_lighting(image, brightness_factor=1.5):
    "this function simulates the lighting conditions by adjusting brightness"
    #brightness factor adjusts the contrast of the image. It MULTIPLIES every pixel value in the image by this factor.
    #if 1, pixels unchanged, >1 increases contrast, <1 decreases contrast
    #beta (brightness shift): ADDS this value to each pixel to adjust brightness
    #if 0 no change in brightness, >0 lightens the image, <0 darkens the image
    return cv2.convertScaleAbs(image, alpha=brightness_factor, beta=0)

#edgeCase-3
#motion blur in a specific direction
#description: Objects in motion that are blurred due to fast movement or camera shake.
#example: A moving car, cyclist, or person with a blurred outline.
#effect: YOLO may have difficulty detecting objects when they are motion-blurred, leading to reduced accuracy.
def apply_motion_blur(image, kernel_size=5):
    "motion blur is linear blur in specific direction, simulating the effect of camera or object motion during image capture"
    #create an empty kernel (all zeros)
    kernel = np.zeros((kernel_size, kernel_size))
    #set the middle row of the kernel to ones (this represents the direction of motion)
    kernel[kernel_size//2, :] = np.ones(kernel_size)
    #kernel[:, kernel_size//2] = np.ones(kernel_size) for vertical
    return cv2.filter2D(image, -1, kernel)

#edgeCase-4
#background noise
#description: Objects appearing in noisy or low-quality images.
#example: A noisy image with lots of pixel artifacts or poor resolution.
#effect: YOLO may detect false positives or miss the correct object due to poor image quality.
def apply_noise(image, noise_factor=0.02):
    "this function adds Gaussian noise to the image"
    row, col, ch = image.shape
    #provide mean (0:no bias), std_dev (spread of noise), size
    #gauss has values sampled from a normal distribution between -1 and 1, multiplying by 255 scales the noise to a reasonable range for pixel values 
    #(i.e., between -255 and 255).
    gauss = np.random.normal(0, noise_factor ** 0.5, (row, col, ch))
    #clip ensures that after adding the noise, all pixel values stay within the valid range of 0 to 255 (8 bit). 
    noisy = np.uint8(np.clip(image + gauss * 255, 0, 255))
    return noisy

#edgeCase-5
#scale variations
#description: Objects at different scales (sizes).
#example: The same object in a close-up versus a distant view.
#effect: YOLO may have difficulty detecting objects that are very large or very small in different image scales.
def apply_scale(image, scale_factor=0.5):
    "this function scales the image by a given factor"
    h, w = image.shape[:2]
    scaled_image = cv2.resize(image, (int(w * scale_factor), int(h * scale_factor)))
    return scaled_image

#edgeCase=6
#rotation
#description: Objects that are rotated in the image.
#example: A car tilted at an angle or a rotated billboard.
#effect: YOLO may struggle to correctly identify objects if they are rotated, especially when not trained on rotated images.
def apply_rotation(image):
    h,w,ch = image.shape
    center = (w//2,h//2)
    rotation_matrix = cv2.getRotationMatrix2D((w//2,h//2),45,0.5)
    #determine the new bounding dimensions to prevent cropping
    #determining the amount by which the horizontal axis (x-axis) is scaled due to the rotation
    cos = np.abs(rotation_matrix[0, 0]) 
    #determining the amount by which the vertical axis (y-axis) is scaled due to the rotation
    sin = np.abs(rotation_matrix[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))
    #adjust the rotation matrix to take the translation into account
    rotation_matrix[0, 2] += (new_w / 2) - center[0]
    rotation_matrix[1, 2] += (new_h / 2) - center[1]
    #perform the affine transformation (rotation) with border handling
    rotated_image = cv2.warpAffine(image, rotation_matrix, (new_w, new_h), borderMode=cv2.BORDER_CONSTANT)
    return rotated_image

#edgeCase-7
#small objects (distant or tiny Objects) objects of various sizes in the same image.
#description: Objects at different scales (sizes).
#example: The same object in a close-up versus a distant view.
#effect: YOLO may have difficulty detecting objects that are very large or very small in different image scales.
def apply_small_objects(image, scale_factor=0.1):
    "this function simulates small objects by resizing the object to a smaller scale"
    h, w = image.shape[:2]
    small_object = cv2.resize(image, (int(w * scale_factor), int(h * scale_factor)))
    #generate random coordinate to place the top-left corner of the small object
    small_object_x = np.random.randint(0, w - small_object.shape[1])
    small_object_y = np.random.randint(0, h - small_object.shape[0])
    #slice the original image[y:y+h,x:x+w] to place the small object
    image[small_object_y:small_object_y + small_object.shape[0], small_object_x:small_object_x + small_object.shape[1]] = small_object
    return image

#edgeCase-8
#scale variations (objects of various sizes in the same image)
#description: Objects at different scales (sizes).
#example: The same object in a close-up versus a distant view.
#effect: YOLO may have difficulty detecting objects that are very large or very small in different image scales.
def apply_scale_variations(image, scales=[0.1, 0.2, 0.8,1.2]):
    "this function simulate scale variations by resizing different parts of the image"
    h, w = image.shape[:2]
    for scale in scales:
        resized_image = cv2.resize(image, (int(w * scale), int(h * scale)))
        if resized_image.shape[1] < w and resized_image.shape[0] < h: #resized image must be smaller than the original image
            x_offset = np.random.randint(0, w - resized_image.shape[1])
            y_offset = np.random.randint(0, h - resized_image.shape[0])
            image[y_offset:y_offset + resized_image.shape[0], x_offset:x_offset + resized_image.shape[1]] = resized_image
        else:
            #resize the image further or handle the case when it doesn't fit
            resized_image = cv2.resize(image, (w // 2, h // 2))  # or any other appropriate size
    return image

#edgeCase-9
#description: Objects with unusual aspect ratios (long and thin or very wide objects).It is w/h (if >1:landscape, if<1:
#portrait, if 1: square)
#example: A long, stretched truck in a traffic scene or a very narrow pillar.
#effect: YOLO might not detect such objects properly if the aspect ratio varies significantly from the training dataset.
#aspect ratio variations (unusual proportions)
def apply_aspect_ratio_variations(image, stretch_factor=1.5):
    "this function simulates aspect ratio (w,h) variations by stretching/compressing the image"
    h, w = image.shape[:2]
    resized_image = cv2.resize(image, (w, int(h*stretch_factor)))  # Stretching width
    return resized_image

#edgeCase-10
#object deformation (distorted object)
#description: Objects that change shape or get distorted.
#example: A crumpled or deformed cardboard box.
#effect: Detection may fail if the object is not represented well in the training data, or if the deformation is substantial.
def apply_object_deformation(image):
    "this function simulates object deformation by applying a warp (distortion)"
    #consider an example
    #before transformation: top left (60,120), top right (460,120), bottom left (60,920), bottom right (460,920)
    #after transformation: top left (60,120), top right (460,100) (moves vertically UP)
    #after transformation: bottom left (50,920) (shifts horizontally LEFT), bottom right (460,900) (moves vertically UP because
    #120-100 =20 so 920-20=900)
    rows, cols, ch = image.shape
    print('rows:height, columns:width', rows,cols)
    pts1 = np.float32([[0, 0], [cols - 1, 0], [0, rows - 1]])
    pts2 = np.float32([[0, 0], [cols-1, 0], [1000, rows-1]])
    matrix = cv2.getAffineTransform(pts1, pts2)
    deformed_image = cv2.warpAffine(image, matrix, (cols, rows))
    return deformed_image

#edgeCase-11
#description: Objects that are close to each other, potentially overlapping.
#example: Several cars in a parking lot, with little space between them.
#effect: YOLO might struggle to distinguish between objects that are too close or overlapping.
def apply_close_proximity_objects(image, object_bbox=(30,250,540,340), scale=0.4, proximity=100):
    #extract the object's region of interest (ROI) using the bounding box (use paint to get coordinates)
    image_resize = cv2.resize(image,(600,600))
    x, y, w, h = object_bbox
    obj_roi = image_resize[y:y+h, x:x+w]
    #resize the ROI to a smaller scale (depends on the object in image otherwise you can keep the sacle as it is)
    object_resized = cv2.resize(obj_roi,(int(w*scale),int(h*0.3)))
    h_new,w_new = object_resized.shape[:2]
    direction = np.random.choice(['right','above'])
    
    if direction == 'right':
        print('right')
        x_new = x + proximity - 30
        y_new = y - 20
    else:
        print('above')
        x_new = x 
        y_new = y - proximity 
        
    print('x_new, w_new, w',x_new,w_new,w)
    print('y_new, h_new, w',y_new,h_new,h) 
    
    print('x_new + w_new',x_new + w_new,w)
    print('y_new + h_new',y_new + h_new,h)
    if x_new + w_new < w and  y_new + h_new < h:
        print('here')
        image[y_new:y_new+h_new,x_new:x_new+w_new] = object_resized   
    return image

#edgeCase-12
#description: Objects that blend into or have a complex background.
#example: A person in a crowd or an object against a similar color background (like a green car in a forest).
#effect: YOLO may misclassify or fail to detect objects due to the complexity of the background.
def apply_clutter(image,num_of_clutter=5):
    h,w = image.shape[:2]
    for _ in range(num_of_clutter):
        clutter = np.random.randint(0,256,(int(w//4),int(h//2),3),dtype=np.uint8)
        h_new,w_new = clutter.shape[:2]
        x = np.random.randint(0,w-w_new)
        y = np.random.randint(0,h-h_new)
        if h_new<h and w_new<w:
            image[y:y+h_new,x:x+w_new] = clutter
    return image

#edgeCase-13
#adverse weather conditions: rain, snow
#description: Environmental factors like rain, snow, fog, or dust that obscure objects.
#example: A car on a foggy road or a pedestrian walking in heavy rain.
#effect: YOLO may fail to detect objects due to reduced visibility and changes in the image texture.
def apply_rain_effect(image, intensity=0.1):
    h, w = image.shape[:2]
    #create a 2D array of random integers between 0 and 256, with the same height and width as the image
    #create a mask by comparing each pixel in the noise array to a threshold 
    #intensity: probability threshold for determining which pixels will be part of the rain effect but on average, 
    #for an intensity of 0.2, around 20% of the pixels will be part of the rain.
    rain = np.random.randint(0, 256, (h, w), dtype=np.uint8) < (intensity * 256)
    #set the pixel values at all 'True' positions in the rain mask to [255, 255, 255] (white), simulating raindrops.
    image[rain] = [255, 255, 255]  #white rain list[]
    return image
def apply_snow_effect(image, intensity=0.005):
    h, w = image.shape[:2]
    num_snowflakes = int(h * w * intensity)
    for _ in range(num_snowflakes):
        #random position, ensuring the snowflake stays within bounds
        x = np.random.randint(0 + 4, w - 4)  #x must be at least radius (4) away from the right edge
        y = np.random.randint(0 + 4, h - 4)  #y must be at least radius (4) away from the bottom edge
        size = np.random.randint(1, 5)  #random size for each snowflake (1 to 4 pixels)
        cv2.circle(image, (x, y), size, (255, 255, 255), -1)  #draw the snowflake (circle)
    return image

#edgeCase-14
#description: Objects on surfaces that cause reflections or glare.
#example: Cars with shiny paint reflecting the surrounding environment or glass surfaces.
#effect: The model might detect reflections as objects, leading to false positives.
def apply_reflective_surface(image):
    h,w = image.shape[:2]
    glare = np.full((h//2,w//2),255,dtype=np.uint8)
    h_new, w_new = glare.shape[:2]
    x = np.random.randint(0,w-w_new)
    y = np.random.randint(0,h-h_new)
    if x+w_new<w and y+h_new<h:
        cv2.rectangle(image,(x,y),(x+h_new, y+w_new),(255,255,255),-1)
    else:
        print('glare too big for the image!')
    return image

#edgeCase-15
#description: Objects appearing in different colors in different positions.
#example: An object of differnt colors, like red, blue, brown.
#effect: YOLO might fail to detect objects if they look substantially different from what the model was trained on
def apply_unexpected_objects(image, num_of_objects=5):
    h,w = image.shape[:2]
    object_size = np.random.randint(100,150)
    for _ in range(num_of_objects):
        x = np.random.randint(0,w-object_size)
        y = np.random.randint(0,h-object_size)
        color = np.random.randint(0,256,(3,)).tolist()
        image[y:y+object_size,x:x+object_size] = color
    return image

#edgeCase-16
#class imbalance (disproportionate class distribution in training)
#description: The dataset has a disproportionate number of objects from different classes.
#example: More images of pedestrians and fewer of cars in a traffic scene dataset.
#effect: YOLO may overfit to the more frequent class, leading to poor performance on the less frequent class.
def apply_class_imbalance(image, class_counts=[80, 5, 5]):
    "This function simulates class imbalance by adding more instances of one class"
    #for example, more instances of the first class
    for _ in range(class_counts[0]):
        #add the first class object (e.g., a car) multiple times
        image = apply_small_objects(image, scale_factor=0.1)  # Smaller instances of car
    for _ in range(class_counts[1]):
        #add the second class object (e.g., a tree) fewer times
        image = apply_scale_variations(image)
    return image

#select the edge case for an image
def select_edgecase(image, edge_case):
    "switch-case logic to apply different transformations based on the edge case"
    if case_type == 'occlusion':
        return apply_occlusion(image)
    elif case_type == 'lighting':
        return apply_lighting(image)
    elif case_type == 'motion_blur':
        return apply_motion_blur(image)
    elif case_type == 'noise':
        return apply_noise(image)
    elif case_type == 'scale':
        return apply_scale(image)
    elif case_type == 'rotation':
        return apply_rotation(image)
    elif case_type == 'small_objects':
        return apply_small_objects(image)
    elif case_type == 'scale_objects':
        return apply_scale_variations(image)
    elif case_type == 'aspect_ratio':
        return apply_aspect_ratio_variations(image)
    elif case_type == 'object_deformation':
        return apply_object_deformation(image)
    elif case_type == 'close_proximity_objects':
        return apply_close_proximity_objects(image)
    elif case_type == 'clutter':
        return apply_clutter(image)
    elif case_type == 'rain':
        return apply_rain_effect(image)
    elif case_type == 'snow':
        return apply_snow_effect(image)
    elif case_type == 'reflective_surface':
        return apply_reflective_surface(image)
    elif case_type == 'unexpected_objects':
        return apply_unexpected_objects(image)
    else:
        return image
    

In [2]:
#select the model YOLOv8 from YOLO
import cv2
from ultralytics import YOLO
model = YOLO("yolov8n.pt")

#load the test image
image = cv2.imread('D:\ComputerVision_Projects\ObjectDetection_YOLOv8\Edge_Test_Cases\input\car.jpg')
    
#selct an edge case to test
case_type = 'clutter' # Change to other types like 'occlusion', 'motion_blur', etc.
    
#apply transformation based on the edge case
transformed_image = select_edgecase(image, case_type)
    
#run YOLO object detection on the transformed image
#save the results in the below path
save_path =r'D:\ComputerVision_Projects\ObjectDetection_YOLOv8\Edge_Test_Cases\output'
#perform the inference on the pre-trained weights
model.predict(transformed_image, save = True, project=save_path,conf=0.5, iou=0.4) 
  
#visualize the transformed image
cv2.imshow('Transformed Image', transformed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

WARNING  Python>=3.10 is required, but Python==3.8.8 is currently installed 

0: 384x640 1 car, 85.6ms
Speed: 4.4ms preprocess, 85.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to D:\ComputerVision_Projects\ObjectDetection_YOLOv8\Edge_Test_Cases\output\predict18


In [ ]:
#results at confidence score 0.5 and IoU at 0.4
#original image - 0.88

# 1. occlusion - 0.87
# 2. lighting - 0.88
# 3. motion_blur - 0.77
# 4. noise - 0.76
# 5. scale - 0.88
# 6. rotation - 0.74
# 7. small_objects - 0.81, did not detect the small car at scale 0.1 of the original image
# 8. scale_objects at scale [0.1, 0.2, 0.8,1.2] - predicted truck (wrong) with 0.74 and did not predict other cars
# 9. aspect_ratio - 0.65
# 10. object_deformation - predicted airplane (wrong) with 0.54
# 11. background_clutter - 0.56
# 12(A) multiple_objects_in_close_proximity (right) - 'Right' car with 0.66 and original car with 0.84
# 12(b) multiple_objects_in_close_proximity (above) - 'Above' car with 0.82 and original car with 0.87
# 13. class_imbalance 
# 14(A). rain - predicted truck (wrong) with 0.78
# 14(B). snow - no prediction
# 15. reflective_surface - predicted a part of the car with 0.93 and other part as truck with 0.66
# 16. unexpected_objects (color variation) - 0.83

In [ ]:
#observations
# 1. strong performance on occlusion and lighting variations
# 2. challenges with motion blur, noise, and rotation - may need more training dataset with these varitions/data augmentation
# 3. difficulty detecting small objects and scale variations - small objects are often challenging for YOLO models
# 4. aspect ratio and object deformation sensitivity 

In [ ]:
#suggestion
# 1. data augmentation - Ideal for increasing dataset size and generalizing the model.
# 2. fine tuning
# 3. hyperparameter tuning such as IoU, confidence score
# 4. other architectures designed for small object detection such as (a) YOLOv8’s smaller scale layers or (b) Faster R-CNN with 
#Feature Pyramid Networks (FPN)
# 5. post-processing adjustments - for missed detections (scale variations) adjust NMS thresholds
#to refine detections where objects are close in scale or overlap significantly
# 6. anchor box adjustments
# 7. synthetic data (GAN) - Ideal for rare or extreme scenarios that are hard to collect.
# 8. ensemble models